In [64]:
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import string

import nltk 
from nltk import pos_tag
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import movie_reviews, stopwords,wordnet
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from rake_nltk import Rake

import numpy as np
from numpy import arange,atleast_2d,argsort

from collections import defaultdict

from surprise import Reader, Dataset, SVD, accuracy
from surprise.model_selection import train_test_split

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

from ast import literal_eval

from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
helpfulCutOffRatings = 20
reviewDataPath = 'amazon/vsmall.csv'
metadataPath = 'amazon/vsmall_metadata.json'
topN = 5

In [8]:
def removeUnusedReviewInfo(rdata):
    rdata = rdata.drop(['unixReviewTime','reviewTime','reviewerName'],axis=1)
    return rdata
    
def createReviewerInfo(data):
    reviewerInfo = data[['reviewerID', 'reviewerName']]
    return reviewerInfo

In [9]:
def createUserMovieMatrix(udata,mdata):
    fullData = udata.merge(mdata, on='asin',how='outer')
    userMovieMatrix = fullData.pivot_table(index='reviewerID', columns = 'asin',values = 'overall').reset_index().rename_axis(None, axis=1)
    userMovieMatching = pd.crosstab(index = fullData['reviewerID'], columns=df['asin'],dropna = False)
    userMovieMatrix = userMovieMatrix.replace(np.nan,0)
    userMovieMatrix = userMovieMatrix.drop(['reviewerID'],axis = 1)
    return userMovieMatrix, userMovieMatching

In [10]:
# Classify data into negative or positive
def classifyBaseReviewRating(x):
    if x > 2.5:
        return "positive"
    if x == 2.5:
        return "neutral"
    if x < 2.5:
        return "negative"

In [11]:
def cleanData(r):
    r = str(r)
    if len(r) == 0:
        return ''
    if type(r) == float:
        return ''
    return r

In [12]:
def getWordnetType(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def cleanReview(r):
    r = str(r)
    if len(r) == 0:
        return ''
    if type(r) == float:
        return ''
    r = r.lower()
    #remove puncutation
    r = [eachWord.strip(string.punctuation) for eachWord in r.split(" ") ]
    #remove numbers
    r = [eachWord for eachWord in r if not any(l.isdigit() for l in eachWord)]
    
    #remove stop words
    stop = stopwords.words('english')
    r = [w for w in r if w not in stop]
    
    #remove words which are empty
    r = [w for w in r if len(w) >0]
    
    #add pos_tags
    pos_tags = pos_tag(r)
    
    #word lemmatize
    r = [WordNetLemmatizer().lemmatize(t[0], getWordnetType(t[1])) for t in pos_tags]
    
    #remove useless words
    r = [w for w in r if len(w) > 1]
    
    #recreate review
    r = " ".join(r)
    return r

# get compound score from VADER and append it as review rating
def getSentimentScore(rdata):
    sa = SentimentIntensityAnalyzer()
    rdata["sentiments"] = rdata["reviewText"].apply(lambda x: sa.polarity_scores(x))
    rdata = pd.concat([rdata.drop(['sentiments'], axis=1), rdata['sentiments'].apply(pd.Series)], axis=1)
    rdata['helpful'] = rdata['helpful'].apply(lambda x: getHelpfulScore(x))
    #compound score from VADER is from -1 to +1. Scaling it to 0 to 5 and multiplying it with helpfulness score
    rdata['reviewRating'] = (1+(rdata['compound']+1)*2)
#     rdata['reviewRating'] = (1+(rdata['compound']+1)*2) * rdata['helpful']
    rdata = rdata.drop(['compound','pos','neg','neu'],axis = 1)
    rdata = rdata.drop(['reviewText','helpful'],axis=1)
    return rdata
    

In [13]:
def getHelpfulScore(x):
    if x == None:
        return 1.0
    x = literal_eval(x)
    a = float(x[0])
    b = float(x[1])

    if b != 0 and a + b > helpfulCutOffRatings-1:
        return a / (a + b)
    else:
        return 1.0

In [14]:
def generateSentimentScoreOnReviews(rdata):
    reviewerInfo = createReviewerInfo(rdata)
    rdata = removeUnusedReviewInfo(rdata)
    rdata["reviewType"] = rdata["overall"].apply(classifyBaseReviewRating)
    rdata['reviewText'] = rdata['reviewText'] + rdata['summary']
    rdata = rdata.drop(['summary'],axis = 1)
    rdata['reviewText'] = rdata['reviewText'].apply(cleanData)    
    rdata['reviewText'] = rdata['reviewText'].apply(lambda x:cleanReview(x))
    rdata = getSentimentScore(rdata)
    return rdata


In [15]:
def getMovieData(path):
    return pd.read_json(path,lines=True,typ='frame')

def preProcessMovieData(moviedata):
    moviedata.sort_values("asin",inplace=True)
    moviedata.drop_duplicates(subset='asin',keep=False,inplace = True)
    moviedata = moviedata.reset_index(drop=True)
    moviedata = moviedata.drop(['rank','main_cat','image','also_buy','also_view','price','details','feature','date','tech1'],axis = 1,errors='ignore')
    return moviedata



In [16]:
def getCosineSimilarity(moviedata):
    count = CountVectorizer()
    count_matrix = count.fit_transform(moviedata['cleanText'])
    # generating the cosine similarity matrix
    cosine_sim = cosine_similarity(count_matrix, count_matrix)
    return cosine_sim

# def getMovieSimilarityFromPlot(moviedata):
#     # initializing the new column
#     moviedata['impWords'] = ''
#     moviedata['cleanText'] = ''
#     for index, movie in moviedata.iterrows():
#         tot = ''
#         description = movie['description']
#         if description != None:
#             if type(description) != float:
#                 if len(description) != 0:
#                     for i in description:
#                         if type(i) != str:
#                             print(description)
#                         tot = tot + i

#         #remove "Movies & TV", "Genre for Featured Categories", "Movies", "Independently Distributed", "All Titles", "All", "4-for-3 DVD", 
#         ignoreWords = ["Movies & TV", "Genre for Featured Categories", "Movies", "Independently Distributed", "All Titles", "All", "4-for-3 DVD"]
#         category = movie['category']
#         if category != None:
#             if type(category) != float:
#                 for w in category:
#                     if w not in ignoreWords:
#                         tot = tot+w
#         brand = movie['brand']
#         if brand != None:
#             if type(brand) != float:
#                 tot = tot + i
#         r = Rake()
#         r.extract_keywords_from_text(tot)
#         # getting the dictionary whith key words as keys and their scores as values
#         key_words_dict_scores = r.get_word_degrees()

#         # assigning the key words to the new column for the corresponding movie
#         movie['impWords'] = list(key_words_dict_scores.keys())
#         movie['cleanText'] = " ".join(map(str,movie['impWords']))
#     moviedata = moviedata.drop(['category','brand','description','impWords'],axis = 1)
#     cosine_sim =  getCosineSimilarity(moviedata)
#     moviedata = moviedata.drop(['cleanText'],axis = 1)
#     return cosine_sim, moviedata


In [17]:
def getMovieAvgRating(moviedata,rdataf):
    movieAvgRating = rdataf.groupby('asin')['overall'].agg(['mean', 'median', 'size'])
    movieAvgRating.columns = ['rating_mean', 'rating_median', 'num_ratings']
    return movieAvgRating


In [18]:
def get_top3_recommendations(predictions, topN = 3):
     
    top_recs = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_recs[uid].append((iid, est))
     
    for uid, user_ratings in top_recs.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_recs[uid] = user_ratings[:topN]
     
    return top_recs

def precision_recall_at_k(predictions, k=10, threshold=3.5):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls


In [19]:
def getPredictionFromMovieContent(moviedata, rdata, testSplit):
    moviedata = preProcessMovieData(moviedata)
    # craete cosine similarity matrix
    cosine_sim, moviedata = getMovieSimilarityFromPlot(moviedata)
    rdata, testdata = train_test_split(rdata, test_size=testSplit)
#     userMovieMatrix = createUserMovieMatrix(rdata,moviedata)
    rdata = rdata.merge(moviedata, on='asin',how='left')
    movieAvgRating = getMovieAvgRating(moviedata,rdata)
    movieAvgRating.columns = ['rating_mean', 'rating_median', 'num_ratings']
    moviedata = moviedata.merge(movieAvgRating, on='asin',how='left')
    
    predictions = getUserRatingPredictionFromMovieContent(cosine_sim, rdata, testdata, moviedata)
    return predictions

In [20]:
def getUserRatingPredictionFromMovieContent(cosine_sim, rdata, testdata, moviedata):
    userMovieMatrix,userMovieMatching = createUserMovieMatrix(rdata, moviedata)
    sumUserMovieWeight = np.dot(userMovieMatrix, cosine_sim)
    sumSimFoRM2M = np.dot(userMovieMatching, cosine_sim)
    userMoviePredict = sumUserMovieWeight/sumSimFoRM2M
    
    predictions = dict()
    
    return predictions

In [21]:
# Based on field in and type
def getSurpriseRatingPrediction(data, ratingCol, testSplit, algo, userBased):
    ratings_dict = {'itemID': list(data.asin),
                'userID': list(data.reviewerID),
                'rating': list(data[ratingCol])}
    df = pd.DataFrame(ratings_dict)
    
    # A reader is still needed but only the rating_scale param is required.
    # The Reader class is used to parse a file containing ratings.
    reader = Reader(rating_scale=(0.5, 5.0))
    # The columns must correspond to user id, item id and ratings (in that order).
    data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)
    trainset, testset = train_test_split(data, test_size=testSplit)
    if algo == 'svd':
        algo = SVD()
    elif algo == 'knn':
        sim_options = {'name': 'cosine',
               'user_based': userBased
               }
        algo = algo = KNNBaseline(sim_options=sim_options)
    algo.fit(trainset)
    print('mean',trainset.global_mean)
    predictions = algo.test(testset)
    return predictions
    

In [22]:
def getPredictions(rdata, moviedata, testSplit, topN):
    # get SVD predictions
    svdPredictions = getSurpriseRatingPrediction(rdata, 'overall', testSplit, 'svd', False )
    
    #get Knn user based predictions based on ratings
    userBasedKNNPredictions = getSurpriseRatingPrediction(rdata, 'overall', testSplit, 'knn', True)
    
    #get Knn movie based predictions based on ratings
    ratingBasedKNNPredictions = getSurpriseRatingPrediction(rdata, 'overall', testSplit, 'knn', False)
    
    #get Knn movie based predictions based on reviews
    rdata = generateSentimentScoreOnReviews(rdata)
    reviewBasedKNNPredictions = getSurpriseRatingPrediction(rdata, 'reviewRating', testSplit, 'knn', False)
    
    #get content based predictions
    contentBasedPredictions = getPredictionFromMovieContent(moviedata, rdata, testSplit)
    
    topNRec = getFinalRec(svdPredictions, userBasedKNNPredictions,
                                           ratingBasedKNNPredictions, reviewBasedKNNPredictions,
                                          contentBasedPredictions, topN)
    return topNRec
    
    

In [23]:
# Combine all the predictions score by weighted average
def combinePredictions(svdPredictions, userBasedKNNPredictions,
                                           ratingBasedKNNPredictions, reviewBasedKNNPredictions,
                                          contentBasedPredictions):
    svdPredictionsNew = defaultdict(list)
    for uid, iid, true_r, est, _ in svdPredictions:
            svdPredictionsNew[uid].append((iid, est))
    
    
    userBasedKNNPredictionsNew = defaultdict(list)
    for uid, iid, true_r, est, _ in userBasedKNNPredictions:
            userBasedKNNPredictionsNew[uid].append((iid, est))
            
    
    ratingBasedKNNPredictionsNew = defaultdict(list)
    for uid, iid, true_r, est, _ in ratingBasedKNNPredictions:
            ratingBasedKNNPredictionsNew[uid].append((iid, est))
    
    
    reviewBasedKNNPredictionsNew = defaultdict(list)
    for uid, iid, true_r, est, _ in reviewBasedKNNPredictions:
            reviewBasedKNNPredictionsNew[uid].append((iid, est))
    
    # contentBasedPredictions is already of the form like above
    # TODO: add scores of different methods and recommend
    
    return top_recs
    

In [25]:
def getFinalRec(svdPredictions, userBasedKNNPredictions,
                                           ratingBasedKNNPredictions, reviewBasedKNNPredictions,
                                          contentBasedPredictions, topN):
    
    predictions = combinePredictions(svdPredictions, userBasedKNNPredictions,
                                           ratingBasedKNNPredictions, reviewBasedKNNPredictions,
                                          contentBasedPredictions)
    
    for uid, user_ratings in top_recs.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_recs[uid] = user_ratings[:topN]
    return top_recs
    

In [ ]:
# m2mCosim = pd.DataFrame(cosine_sim)
# indexWithMovieId = moviedata['asin']
# m2mCosim.columns = [str(indexWithMovieId[int(col)]) for col in m2mCosim.columns]
# m2mCosim.index = [indexWithMovieId[idx] for idx in m2mCosim.index]
# # m2mCosim.iloc[0].sort_values(ascending=False)[:10]
# userRatings = rdataf[rdataf.reviewerID == 'A1GHUN5HXMHZ89']
# userRatings.head(2)
# userDataWithScore = moviedata.reset_index().merge(userRatings, on='asin')

# userDataWithScore['weight'] = userDataWithScore['overall']/5
# # df_user_data_with_tags['weight'].values
# # df_user_data_with_tags
# user_profile = np.dot(count_matrix[userDataWithScore['index'].values].toarray().T, userDataWithScore['weight'].values)
# C = cosine_similarity(atleast_2d(user_profile), count_matrix)
# R = argsort(C)[:, ::-1]
# recommendations = [i for i in R[0] if i not in userDataWithScore['index'].values]
# rdataf['asin'][recommendations].head(10)


In [155]:
rdata = pd.read_csv('amazon/vsmall.csv', names=['reviewerID','asin','reviewerName','helpful','reviewText','overall','summary','unixReviewTime','reviewTime'], skiprows=1, sep='\t')


In [156]:
rdata.reset_index(inplace=True)

In [157]:
rdata = rdata.drop(['index'],axis =1)

In [158]:
moviedata = pd.read_json('amazon/vsmall_metadata.json',lines=True,typ='frame')

In [159]:
moviedata = preProcessMovieData(moviedata)
moviedata = getMovieSimilarityFromPlot(moviedata)
moviedata

,title,asin,cleanText
0,Rise and Swine (Good Eats Vol. 7),0000143502,corn dogs macerated strawberries good eats vol...
1,My Fair Pastry (Good Eats Vol. 9),0000143529,southern biscuits salmon turnovers fruit tart ...
2,"Everyday Italian (with Giada de Laurentiis), V...",0000143561,christmas dinner party birthday giada de laure...
3,"Barefoot Contessa (with Ina Garten), Entertain...",0000143588,prepare scrumptious dishes delicious offerings...
4,Understanding Seizures and Epilepsy,0000695009,
...,...,...,...
277,The Deep End of the Ocean VHS,0767819519,careful certainly credible part 3 expect every...
278,Pal Joey,0767821807,charming nightclub singer title character danc...
279,Hard Times,0767824555,break subject directed violence spare existent...
280,Monty Python and the Holy Grail,0767824571,short list almost certainly dead kind pythons ...


In [160]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf = TfidfVectorizer()
moviesTFIDFdescribed = tf_idf.fit_transform(moviedata.cleanText)

In [161]:
# userAsinRating = rdata[['reviewerID','asin','overall']]
# userAsinRating

In [162]:
userAsinRating = rdata[['reviewerID','asin','overall']]
userMoviePlot = pd.merge(userAsinRating,moviedata,on='asin',how='left')

In [163]:
userMoviePlot = userMoviePlot.replace(np.nan, '', regex=True)

In [164]:
def combineMovieText(text):
    text_as_str = ' '.join(set(text))
    return text_as_str

In [165]:
userMoviePlot = userMoviePlot.groupby('reviewerID')['cleanText'].agg(combineMovieText)

In [166]:
userMoviePlot.name = 'cleanText'
userMoviePlot = userMoviePlot.reset_index()

In [167]:
userMoviePlot

,reviewerID,cleanText
0,A101FH66IRVOEQ,
1,A102RDJLOHWS0W,
2,A1033RWNZWEMR5,
3,A103EXN5Q7HX6Z,
4,A106016KSI0YQ,
...,...,...
1876,AZT60HLHFL0V0,
1877,AZWJ4ZNG2Z5GO,
1878,AZXGPM8EKSHE9,
1879,AZXHK8IO25FL6,


In [168]:
userAsinRating

,reviewerID,asin,overall
0,A22XGVAAD5WCNS,6304217420,5
1,A2Z5RFU4DH3T0G,B00005JNBQ,5
2,AR9OIGY3T6EU7,B000ZECQ08,4
3,AEVIYGYO0FEOM,6300270351,4
4,AQUNXYDV20Z9R,630447976X,5
...,...,...,...
1995,A8IZXBE7J6RNJ,6302642248,5
1996,A1RKS0O04U3GY3,B00023P4HY,4
1997,A3RICVFW2YD1LZ,B000FILVFA,2
1998,ANCOMAI0I7LVG,B0062NAWH0,3


In [169]:
# userAsinRatingPlot = pd.merge(userAsinRating,userMoviePlot, on='reviewerID',how='left')
userDataWithTextP =  userAsinRating.reset_index().merge(userMoviePlot,on='reviewerID')
userDataWithTextP

,index,reviewerID,asin,overall,cleanText
0,0,A22XGVAAD5WCNS,6304217420,5,
1,1,A2Z5RFU4DH3T0G,B00005JNBQ,5,
2,2,AR9OIGY3T6EU7,B000ZECQ08,4,
3,3,AEVIYGYO0FEOM,6300270351,4,
4,4,AQUNXYDV20Z9R,630447976X,5,
...,...,...,...,...,...
1995,1994,A8P076YJHIMBN,B0002NRRQU,4,
1996,1995,A8IZXBE7J6RNJ,6302642248,5,
1997,1996,A1RKS0O04U3GY3,B00023P4HY,4,
1998,1997,A3RICVFW2YD1LZ,B000FILVFA,2,


In [170]:
userDataWithTextP['weight'] = userDataWithTextP['overall']/5
userDataWithTextP

,index,reviewerID,asin,overall,cleanText,weight
0,0,A22XGVAAD5WCNS,6304217420,5,,1.0
1,1,A2Z5RFU4DH3T0G,B00005JNBQ,5,,1.0
2,2,AR9OIGY3T6EU7,B000ZECQ08,4,,0.8
3,3,AEVIYGYO0FEOM,6300270351,4,,0.8
4,4,AQUNXYDV20Z9R,630447976X,5,,1.0
...,...,...,...,...,...,...
1995,1994,A8P076YJHIMBN,B0002NRRQU,4,,0.8
1996,1995,A8IZXBE7J6RNJ,6302642248,5,,1.0
1997,1996,A1RKS0O04U3GY3,B00023P4HY,4,,0.8
1998,1997,A3RICVFW2YD1LZ,B000FILVFA,2,,0.4


In [172]:
# moviesTFIDFdescribed[userDataWithTextP['index'].values]

IndexError: index (1999) out of range

In [ ]:
user_profile = np.dot(df_movies_tf_idf_described[df_user_data_with_tags['index'].values].toarray().T, df_user_data_with_tags['weight'].values)



In [29]:
def getMovieSimilarityFromPlot(moviedata):
    # initializing the new column
    moviedata['impWords'] = ''
    moviedata['cleanText'] = ''
    for index, movie in moviedata.iterrows():
        tot = ''
        description = movie['description']
        if description != None:
            if type(description) != float:
                if len(description) != 0:
                    for i in description:
                        if type(i) != str:
                            print(description)
                        tot = tot + i

        #remove "Movies & TV", "Genre for Featured Categories", "Movies", "Independently Distributed", "All Titles", "All", "4-for-3 DVD", 
        ignoreWords = ["Movies & TV", "Genre for Featured Categories", "Movies", "Independently Distributed", "All Titles", "All", "4-for-3 DVD"]
        category = movie['category']
        if category != None:
            if type(category) != float:
                for w in category:
                    if w not in ignoreWords:
                        tot = tot+w
        brand = movie['brand']
        if brand != None:
            if type(brand) != float:
                tot = tot + i
        r = Rake()
        r.extract_keywords_from_text(tot)
        # getting the dictionary whith key words as keys and their scores as values
        key_words_dict_scores = r.get_word_degrees()

        # assigning the key words to the new column for the corresponding movie
        movie['impWords'] = list(key_words_dict_scores.keys())
        movie['cleanText'] = " ".join(map(str,movie['impWords']))
    moviedata = moviedata.drop(['category','brand','description','impWords'],axis = 1)
#     cosine_sim =  getCosineSimilarity(moviedata)
#     moviedata = moviedata.drop(['cleanText'],axis = 1)
    return moviedata

In [28]:
def getPredictionFromMovieContent(moviedata, rdata, testSplit):
    moviedata = preProcessMovieData(moviedata)
    # craete cosine similarity matrix
    cosine_sim, moviedata = getMovieSimilarityFromPlot(moviedata)
    rdata, testdata = train_test_split(rdata, test_size=testSplit)
#     userMovieMatrix = createUserMovieMatrix(rdata,moviedata)
    rdata = rdata.merge(moviedata, on='asin',how='left')
    movieAvgRating = getMovieAvgRating(moviedata,rdata)
    movieAvgRating.columns = ['rating_mean', 'rating_median', 'num_ratings']
    moviedata = moviedata.merge(movieAvgRating, on='asin',how='left')
    
    predictions = getUserRatingPredictionFromMovieContent(cosine_sim, rdata, testdata, moviedata)
    return predictions

In [ ]:
# craete cosine similarity matrix
# cosine_sim, moviedata = getMovieSimilarityFromPlot(moviedata)
# rdata, testdata = train_test_split(rdata, test_size=testSplit)
# #     userMovieMatrix = createUserMovieMatrix(rdata,moviedata)
# rdata = rdata.merge(moviedata, on='asin',how='left')
# movieAvgRating = getMovieAvgRating(moviedata,rdata)
# movieAvgRating.columns = ['rating_mean', 'rating_median', 'num_ratings']
# moviedata = moviedata.merge(movieAvgRating, on='asin',how='left')

# predictions = getUserRatingPredictionFromMovieContent(cosine_sim, rdata, testdata, moviedata)